In [1]:
import os
import pandas as pd

In [2]:
cat_attrs: str = "f12 f11"
balance_data: bool = False
clear_data: bool = True
n_jobs: int = 1
verbose: int = 10

In [3]:
# Parameters
n_jobs = 8


In [4]:
cat_attrs: list[str] = cat_attrs.split()
cat_attrs

['f12', 'f11']

In [5]:
pd.set_option('display.float_format', '{:.3f}'.format)

In [6]:
csv_path = os.path.join('data', 'data.csv')
data = pd.read_csv(csv_path)

In [7]:
X = data.drop(columns=['target', 'p'])
y = data['target']

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = pd.Series(y, name='target')

In [9]:
from sklearn.model_selection import train_test_split

test_size_ratio = 0.2
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=test_size_ratio, 
    stratify=y, 
    random_state=42
)

print(f"Train: X={X_train.shape} y={y_train.shape}")
print(f"Test: X={X_test.shape} y={y_test.shape}")

Train: X=(838860, 17) y=(838860,)
Test: X=(209715, 17) y=(209715,)


In [10]:
from collections import Counter

if balance_data:
    train = pd.concat([X_train, y_train], axis=1)
    train_n = train.shape[0]
    
    class_counts = Counter(train['target'])
    min_class = min(class_counts, key=lambda x: class_counts[x])
    
    new_train = train.groupby('target').apply(
        lambda x: x.sample(class_counts[min_class])
    ).reset_index(drop=True)
    removed_n = train.shape[0] - new_train.shape[0]
    removed_ratio = removed_n / train_n
    
    print(f"Majority class: '{min_class}'")
    print(f"Records removed: {removed_n} ({removed_ratio * 100:.3f} %)")
    X_train = new_train.drop(columns=['target'])
    y_train = new_train['target']
    print(f"\nTrain: X={X_train.shape} y={y_train.shape}")
else:
    print("balance_data set to False")

balance_data set to False


In [11]:
y_train_pos_ratio = y_train[y_train == 1].shape[0] / y_train.shape[0]
y_test_pos_ratio = y_test[y_test == 1].shape[0] / y_test.shape[0]

print("Train positives ratio:", f"{y_train_pos_ratio * 100:.5f} %")
print("Test positives ratio: ", f"{y_test_pos_ratio * 100:.5f} %")

Train positives ratio: 1.69301 %
Test positives ratio:  1.69325 %


In [12]:
if clear_data:
    train = pd.concat([X_train, y_train], axis=1)
    train_n = train.shape[0]
    
    new_train = train.drop_duplicates()
    duplicates_n = train.shape[0] - new_train.shape[0]
    duplicates_ratio = duplicates_n / train_n
    print(f"Duplicates removed: {duplicates_n} ({duplicates_ratio * 100:.3f} %)")
    train = new_train
    
    new_train = train.drop_duplicates(subset=train.columns.difference(['target']))
    collisions_n = train.shape[0] - new_train.shape[0]
    collisions_ratio = collisions_n / train_n
    print(f"Collisions removed: {collisions_n} ({collisions_ratio * 100:.3f} %)")
    train = new_train
    
    print(f"Total removed:      {duplicates_n + collisions_n} "
          f"({(duplicates_ratio + collisions_ratio) * 100:.3f} %)")
    X_train = train.drop(columns=['target'])
    y_train = train['target']
    print(f"\nTrain: X={X_train.shape} y={y_train.shape}")
else:
    print("clear_data set to False")

Duplicates removed: 61923 (7.382 %)


Collisions removed: 56 (0.007 %)
Total removed:      61979 (7.388 %)

Train: X=(776881, 17) y=(776881,)


In [13]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

cat_encoder = OneHotEncoder().fit(X[cat_attrs])
cat_encoder.categories_  # TODO

[array([0, 1], dtype=int64), array([0, 1], dtype=int64)]

In [14]:
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.pipeline import Pipeline as ImPipeline

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', 'passthrough', cat_attrs)
    ],
    remainder=StandardScaler(),
    verbose_feature_names_out=True
)
preprocessor = preprocessor.set_output(transform='pandas')

one_hot = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), list(map(lambda attr: 'cat__' + attr, cat_attrs)))
    ],
    remainder='passthrough',
    verbose_feature_names_out=True
)

pipeline = ImPipeline(steps=[
    ('preprocessor', preprocessor),
    ('sampling', SMOTENC(categorical_features=list(map(lambda attr: 'cat__' + attr, cat_attrs)), random_state=42)),
    ('one_hot', one_hot),
    ('classifier', LogisticRegression())
])

In [15]:
# z_x, z_y = pipeline.fit_resample(X_train, y_train)
# z_x, z_y

In [16]:
# (z_y == 0).sum(), (z_y == 1).sum()

In [17]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=y_train.unique(), y=y_train)
class_weights = dict(enumerate(class_weights))
class_weights

{0: 0.5087769277216532, 1: 28.98377107894344}

In [18]:
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler

param_grid = [
    {
        # 'one_hot': ['passthrough'],
        # 'classifier__categorical_features': [list(map(lambda attr: 'cat__' + attr, cat_attrs))],

        'classifier': [HistGradientBoostingClassifier(random_state=42)],
        'classifier__learning_rate': [0.1, 1],  # 0.1
        'classifier__max_leaf_nodes': [31, None],  # 31
        'classifier__max_iter': [1000],  # 100
        'classifier__early_stopping': [True],  # True
        'classifier__n_iter_no_change': [30],  # 10
        'classifier__class_weight': [None],
    },
    {
        'one_hot': ['passthrough'],
        'classifier__categorical_features': [list(map(lambda attr: 'cat__' + attr, cat_attrs))],

        'classifier': [HistGradientBoostingClassifier(random_state=42)],
        'classifier__learning_rate': [0.1, 1],  # 0.1
        'classifier__max_leaf_nodes': [31, None],  # 31
        'classifier__max_iter': [1000],  # 100
        'classifier__early_stopping': [True],  # True
        'classifier__n_iter_no_change': [30],  # 10
        'classifier__class_weight': [None],
    },
]

In [19]:
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit

grid_search = GridSearchCV(
    pipeline, param_grid,
    cv=StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42),
    scoring=['f1', 'precision', 'recall'],
    refit='f1',
    return_train_score=True,
    n_jobs=n_jobs,
    verbose=verbose
)

grid_search.fit(X_train, y_train);

Fitting 5 folds for each of 8 candidates, totalling 40 fits


In [20]:
grid_search.best_params_

{'classifier': HistGradientBoostingClassifier(random_state=42),
 'classifier__class_weight': None,
 'classifier__early_stopping': True,
 'classifier__learning_rate': 0.1,
 'classifier__max_iter': 1000,
 'classifier__max_leaf_nodes': 31,
 'classifier__n_iter_no_change': 30}

In [21]:
grid_search.best_estimator_

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('cat', 'passthrough',
                                                  ['f12', 'f11'])])),
                ('sampling',
                 SMOTENC(categorical_features=['cat__f12', 'cat__f11'],
                         random_state=42)),
                ('one_hot',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat', OneHotEncoder(),
                                                  ['cat__f12', 'cat__f11'])])),
                ('classifier',
                 HistGradientBoostingClassifier(early_stopping=True,
                                                max_iter=1000,
                                                n_iter_no_change=30,
                                                random_state=42))])

In [22]:
grid_search.best_score_

0.7271668544396706

In [23]:
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values('rank_test_f1')
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier,param_classifier__class_weight,param_classifier__early_stopping,param_classifier__learning_rate,param_classifier__max_iter,param_classifier__max_leaf_nodes,...,mean_test_recall,std_test_recall,rank_test_recall,split0_train_recall,split1_train_recall,split2_train_recall,split3_train_recall,split4_train_recall,mean_train_recall,std_train_recall
0,179.924,29.963,5.632,1.758,HistGradientBoostingClassifier(random_state=42),None,True,0.100,1000,31,...,0.817,0.015,4,0.856,0.858,0.858,0.854,0.855,0.856,0.001
1,499.651,49.990,8.156,0.375,HistGradientBoostingClassifier(random_state=42),None,True,0.100,1000,None,...,0.785,0.004,5,0.971,0.969,0.972,0.967,0.969,0.970,0.002
5,407.871,9.994,6.328,0.469,HistGradientBoostingClassifier(random_state=42),None,True,0.100,1000,None,...,0.784,0.004,6,0.971,0.969,0.972,0.967,0.970,0.970,0.001
4,99.472,44.953,5.888,2.985,HistGradientBoostingClassifier(random_state=42),None,True,0.100,1000,31,...,0.818,0.019,3,0.856,0.871,0.857,0.856,0.860,0.860,0.006
3,101.744,1.366,2.799,0.052,HistGradientBoostingClassifier(random_state=42),None,True,1,1000,None,...,0.777,0.003,7,0.970,0.969,0.972,0.968,0.970,0.970,0.001
7,94.940,4.412,2.785,0.064,HistGradientBoostingClassifier(random_state=42),None,True,1,1000,None,...,0.776,0.004,8,0.971,0.970,0.973,0.968,0.970,0.970,0.002
2,27.622,0.936,0.605,0.034,HistGradientBoostingClassifier(random_state=42),None,True,1,1000,31,...,0.840,0.009,2,0.857,0.853,0.861,0.862,0.846,0.856,0.006
6,26.667,1.061,0.598,0.055,HistGradientBoostingClassifier(random_state=42),None,True,1,1000,31,...,0.844,0.013,1,0.857,0.852,0.876,0.862,0.844,0.858,0.011


In [24]:
from sklearn.metrics import f1_score, precision_score, recall_score

y_pred = grid_search.best_estimator_.predict(X_test)
f1 = f1_score(y_test, y_pred)
p = precision_score(y_test, y_pred)
r = recall_score(y_test, y_pred)

print("F1:", f1)
print("Precision:", p)
print("Recall:", r)

F1: 0.7675576519916143
Precision: 0.7177162460181328
Recall: 0.8248380737820332


In [25]:
from joblib import dump

if not os.path.exists('out'):
    os.makedirs('out')
dump(grid_search, os.path.join('out', 'GridSearchCV_dump.pkl'), compress=True)

['out\\GridSearchCV_dump.pkl']